In [144]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

  Activating project at `~/Projects/DIVAnd-FAIR-EASE`


In [118]:
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, footprintdir])

2-element Vector{String}:
 "../data"
 "../Footprint"

In [119]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [155]:
datasource = "EMODnet Chemistry"
domain = [12., 18., 43., 46.]
parameter1 = "TEMP"
if datasource == "World Ocean Database"
    parameter1 = "Temperature"
elseif datasource == "EMODnet Chemistry"  
    parameter1 = "ITS_90_water_temperature"
elseif datasource == "SeaDataNet CDI TS"
    parameter1 = "ITS-90 water temperature"
end
parameter1 = "Water_body_phosphate"
parameter2 = "TEMP"
datestart = Dates.Date(1980, 1, 1)
dateend = Dates.Date(2020, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 10.
regionname = "BlackSea"
@info("$(datestart) → $(dateend), $(parameter1)")

[ Info: 1980-01-01 → 2020-01-01, Water_body_phosphate


In [150]:
beaconURL = beacon_services[datasource]
footprintURL = joinpath(beaconURL, "api/datasets/footprint")
datasource_name = replace(datasource, " " => "_")
footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
@info("Footprint endpoint: $(footprintURL)")
@info("Footprint file: $(footprintfile)")

[ Info: Footprint endpoint: https://beacon-emod-chem.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_EMODnet_Chemistry.json


In [139]:
if isfile(footprintfile)
    @info("Footprint file already downloaded")
else
    @info("Writing Footprint file")
    open(footprintfile, "w") do io
        r = HTTP.request("GET", footprintURL, 
            ["Authorization" => "Bearer $(token)"],
            response_stream=io)
        @info(r.status)
    end
end

[ Info: Footprint file already downloaded


In [140]:
# Note 1: the JSON file is 193M, just to get the list of variables?

In [148]:
data = JSON.parsefile(footprintfile);
attributes = data["unique_column_attributes"]
params = sort(collect(keys(attributes)));
parameter1 in params
@show(params)

params = ["Access_ordering_of_data", "Access_restriction", "Alternative_cruise_name", "Alternative_station_name", "Bot_Depth", "Bottle", "Bottle_qc", "C22_flag", "C22_flag_qc", "CDI_partner", "CDI_record_creation_date", "CDI_record_id", "CSR_platform_code", "CSR_platform_name", "CSR_references", "Category", "Codes_in_Originator_File", "Comments", "Cruise_name", "Cruise_start_date", "Data_Distributor", "Data_Holding_centre", "Data_format", "Data_format_version", "Data_set_creation_date", "Data_set_name", "Data_size", "Database_reference", "Datum", "Depth", "Depth_qc", "Depth_reference", "Discipline", "EDMED_references", "EDMO_code", "End_date", "End_time", "FireSeqNo", "FireSeqNo_qc", "Frequency", "Frequency_unit", "ITS_90_water_temperature", "ITS_90_water_temperature_qc", "Instrument_Info", "Instrument_gear_type", "LOCAL_CDI_ID", "Maximum_instrument_depth", "Measuring_area_type", "Minimum_instrument_depth", "Originator", "P35_Contributor_Codes", "Platform_type", "Pressure", "Pressure_q

132-element Vector{String}:
 "Access_ordering_of_data"
 "Access_restriction"
 "Alternative_cruise_name"
 "Alternative_station_name"
 "Bot_Depth"
 "Bottle"
 "Bottle_qc"
 "C22_flag"
 "C22_flag_qc"
 "CDI_partner"
 "CDI_record_creation_date"
 "CDI_record_id"
 "CSR_platform_code"
 ⋮
 "Water_body_urea_qc"
 "Water_depth"
 "cruise_id"
 "date_time"
 "latitude"
 "longitude"
 "pHT"
 "pHT_qc"
 "ph_total_SD_water_measured"
 "ph_total_SD_water_measured_qc"
 "station_id"
 "station_type"

In [156]:
query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
    mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

"{\"query_parameters\":[{\"column_name\":\"Water_body_phosphate\",\"alias\":\"Water_body_phosphate\"},{\"column_name\":\"date_time\",\"alias\":\"TIME\"},{\"column_name\":\"Depth\",\"alias\":\"DEPTH\"},{\"column_name\":\"longitude\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"latitude\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":21549,\"max\":36159},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":10.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0}],\"output\":{\"format\":\"netcdf\"}}"

In [157]:
# Note 2: cannot work with such a long list of parameters!!

In [158]:
filename = joinpath(datadir, "$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
filename

"../data/EMODnet_Chemistry_Water_body_phosphate_BlackSea_19800101-20200101_0-10m.nc"

In [159]:
@info("Data will be written in file:\n$(filename)")

if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-type"=> "application/json",
         "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end

@info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")

┌ Info: Data will be written in file:
└ ../data/EMODnet_Chemistry_Water_body_phosphate_BlackSea_19800101-20200101_0-10m.nc


  0.163361 seconds (24.26 k allocations: 1.268 MiB, 2 lock conflicts, 16.61% compilation time)


[ Info: 204
[ Info: NetCDF file size: 0.0M


In [44]:
query

"{\"query_parameters\":[{\"column_name\":\"TEMP\",\"alias\":\"TEMP\"},{\"column_name\":\"JULD\",\"alias\":\"TIME\"},{\"column_name\":\"DEPH\",\"alias\":\"DEPTH\"},{\"column_name\":\"LONGITUDE\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"LATITUDE\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":21915,\"max\":22645},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":10.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0},{\"for_query_parameter\":\"TEMP\",\"min\":-2.0,\"max\":30.0}],\"output\":{\"format\":\"netcdf\"}}"